Tóm tắt thông tin mô hình:
- Mô hình sử dụng GXBoost
- Kiểm định whole dataset cross-validation: Để tận dụng toàn bộ tập dataset để train mô hình (vì dataset hiện chỉ giới hạn  4k dòng)


## TD forecast on python

In [5]:
%load_ext autoreload
%autoreload 2
import pyodbc
import pandas as pd

import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn import model_selection

server = '10.16.157.42'
database = 'RB_DATA'

connection_string = 'DRIVER={ODBC Driver 17 for SQL Server};SERVER=' + server + ';DATABASE=' + database + ';Trusted_Connection=yes'
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()
sql_query = '''
SELECT PERIOD, TERM_, MOVEMENT_TYPE, MOVEMENT_TYPE_2, CURRENCY_2, EOP_CM, EOP_LM
FROM USER_DATA.HIENNPD3.FCST_EOP_TD_MAINMODEL 
WHERE (PERIOD like '2023%' or PERIOD like '2024%' )
AND (TERM_ IS NOT NULL AND MOVEMENT_TYPE IS NOT NULL AND CURRENCY_2 IS NOT NULL)  
'''
cursor.execute(sql_query)
rows = cursor.fetchall()

columns = [column[0] for column in cursor.description]
df = pd.DataFrame.from_records(rows, columns=columns)

cursor.close()
conn.close() 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
df_v2 = df 
df_v2['PERIOD'] = pd.to_datetime(df_v2['PERIOD']) 
df_v2['YEAR'] = df_v2['PERIOD'].dt.year 
df_v2['MONTH'] = df_v2['PERIOD'].dt.month  
 
df_v2

,PERIOD,TERM_,MOVEMENT_TYPE,MOVEMENT_TYPE_2,CURRENCY_2,EOP_CM,EOP_LM,YEAR,MONTH
0,2023-11-01,From 18M,4UNCHANGED_AUTOROLL,1UNCHANGED,VND,1551.775377,1551.775377,2023,11
1,2023-02-01,From 18M,5NEW_AUTOROLL,2NEW,VND,309.989244,0.000000,2023,2
2,2023-11-01,10-11M,0UNCHANGED_NOTROLL,1UNCHANGED,VND,154.466422,154.466422,2023,11
3,2023-06-01,From 18M,0UNCHANGED_NOTROLL,1UNCHANGED,VND,279.780458,279.780458,2023,6
4,2023-08-01,6-9M,5NEW_AUTOROLL,2NEW,NON-VND,13.878628,0.000000,2023,8
...,...,...,...,...,...,...,...,...,...
1103,2023-01-01,1-3M,5NEW_AUTOROLL,2NEW,VND,6005.009669,0.000000,2023,1
1104,2023-08-01,1-3M,1NEW_NOTROLL,2NEW,NON-VND,0.096390,0.000000,2023,8
1105,2023-06-01,12-18M,9OTHERS,6OTHERS,VND,98.139834,77.158864,2023,6
1106,2023-01-01,From 18M,9OTHERS,6OTHERS,VND,1034.881814,0.000000,2023,1


In [34]:
# df_v2.loc[df_v2['EOP_LM'] == 0, df_v2.columns[2]].unique()

array(['5NEW_AUTOROLL', '1NEW_NOTROLL', '9OTHERS', '6ROLL_AUTOROLL',
       '0UNCHANGED_NOTROLL', '4UNCHANGED_AUTOROLL'], dtype=object)

In [25]:
categorical_cols = [ 'MOVEMENT_TYPE', 'MOVEMENT_TYPE_2', 'TERM_','CURRENCY_2']

fcstP_list = ['EOP_CM', 'EOP_LM']

# Fore fcst_value == 'EOP_LM':
df_v2 = df_v2[~df_v2['MOVEMENT_TYPE_2'].isin(['2NEW', '5CHANGECIF'])]



# #Fore fcst_value == 'EOP_CM':
# df_v2 = df_v2[~df_v2['MOVEMENT_TYPE_2'].isin(['4SCHEDULE', '4PREMATURE'])]

    
# Encode categorical variables using one-hot encoding
df_v2 = pd.get_dummies(df_v2, columns=categorical_cols, drop_first=True)   

X = df_v2.drop(['EOP_CM', 'EOP_LM','PERIOD'], axis=1)
y = df_v2['EOP_CM']
model = xgb.XGBRegressor()
model.fit(X, y)
cv_scores = cross_val_score(model, X, y, cv=15, scoring='r2')
y_pred = cross_val_predict(model, X, y, cv=15)
df_v2[f'FCST_EOP_CM'] = y_pred

df_v2


,PERIOD,EOP_CM,EOP_LM,YEAR,MONTH,MOVEMENT_TYPE_2SCHE_NOTROLL,MOVEMENT_TYPE_3PREMATURE_NOTROLL,MOVEMENT_TYPE_4UNCHANGED_AUTOROLL,MOVEMENT_TYPE_6ROLL_AUTOROLL,MOVEMENT_TYPE_7PREMATURE_AUTOROLL,...,MOVEMENT_TYPE_2_4SCHEDULE,MOVEMENT_TYPE_2_6OTHERS,TERM__10-11M,TERM__12-18M,TERM__4-5M,TERM__6-9M,TERM__From 18M,TERM__less than 1M,CURRENCY_2_VND,FCST_EOP_CM
0,2023-11-01,1551.775377,1551.775377,2023,11,False,False,True,False,False,...,False,False,False,False,False,False,True,False,True,2631.364990
2,2023-11-01,154.466422,154.466422,2023,11,False,False,False,False,False,...,False,False,True,False,False,False,False,False,True,65.809677
3,2023-06-01,279.780458,279.780458,2023,6,False,False,False,False,False,...,False,False,False,False,False,False,True,False,True,321.924774
5,2024-01-01,46220.227206,46220.227206,2024,1,False,False,True,False,False,...,False,False,False,True,False,False,False,False,True,3020.906250
7,2023-05-01,0.000000,20486.517315,2023,5,False,False,False,False,True,...,False,False,False,False,False,False,False,True,True,18.661676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,2024-01-01,202.451804,202.451804,2024,1,False,False,True,False,False,...,False,False,True,False,False,False,False,False,True,208.890976
1102,2023-10-01,36.591351,36.311577,2023,10,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,-38.741272
1105,2023-06-01,98.139834,77.158864,2023,6,False,False,False,False,False,...,False,True,False,True,False,False,False,False,True,669.478027
1106,2023-01-01,1034.881814,0.000000,2023,1,False,False,False,False,False,...,False,True,False,False,False,False,True,False,True,1285.651855


,PERIOD,EOP_CM,EOP_LM,YEAR,MONTH,MOVEMENT_TYPE_2SCHE_NOTROLL,MOVEMENT_TYPE_3PREMATURE_NOTROLL,MOVEMENT_TYPE_4UNCHANGED_AUTOROLL,MOVEMENT_TYPE_6ROLL_AUTOROLL,MOVEMENT_TYPE_7PREMATURE_AUTOROLL,...,MOVEMENT_TYPE_2_4SCHEDULE,MOVEMENT_TYPE_2_6OTHERS,TERM__10-11M,TERM__12-18M,TERM__4-5M,TERM__6-9M,TERM__From 18M,TERM__less than 1M,CURRENCY_2_VND,FCST_EOP_CM
18,2023-09-01,0.000000,0.0,2023,9,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,11.270011
28,2023-01-01,5.662972,0.0,2023,1,False,False,False,True,False,...,False,False,False,False,True,False,False,False,True,-207.061661
52,2023-04-01,0.000000,0.0,2023,4,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,72.710732
59,2023-01-01,221.539843,0.0,2023,1,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,1191.395752
79,2023-10-01,0.000000,0.0,2023,10,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,11.131686
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1020,2023-12-01,0.000000,0.0,2023,12,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,2.443130
1049,2023-01-01,0.000000,0.0,2023,1,False,False,True,False,False,...,False,False,False,False,True,False,False,False,True,-2830.458984
1063,2023-03-01,0.000000,0.0,2023,3,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,95.781837
1075,2023-02-01,0.000000,0.0,2023,2,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,92.341591


In [14]:

# categorical_cols = [ 'movement_type', 'movement_type_2', 'Term_CM','Currency_2']

# # Encode categorical variables using one-hot encoding
# encoded_df = pd.get_dummies(df_v2, columns=categorical_cols, drop_first=True)   

# fcstP_list = ['NET', 'CIE_VOL', 'VOF_VOL']

# r2_result = []

# for  fcst_value in fcstP_list:
#     X = encoded_df.drop([fcst_value,'EOP_CM', 'EOP_LM', 'period', 'TD_type','NET','CIE_VOL', 'ADB_VOL', 'VOF_VOL', 'NII_VOL'], axis=1)
#     y = encoded_df[fcst_value]
#     X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.6, random_state=5)
#     # Tạo và huấn luyện mô hình XGBoost
#     model = xgb.XGBRegressor()
#     model.fit(X_train, y_train)
#     cv_scores = cross_val_score(model, X, y, cv=5)  # Adjust the number of folds as desired
#     df_v2[f'FCST_{fcst_value}'] = model.predict(X)
#     # r2_train_CM = r2_score(y, df_v2[f'FCST_{fcst_value}'])
#     # r2_result.append(r2_train_CM)
#     cv_scores = cross_val_score(model, X, y, cv= 25 , scoring='r2')  # Adjust the number of folds as desired
#     r2_result.append(cv_scores.mean())
# r2_result


[0.2975174100381485, 0.9916877482035217, 0.9905550359611383]

In [51]:
import pandas as pd
from sklearn.model_selection import cross_val_score, cross_val_predict
import xgboost as xgb

categorical_cols = ['movement_type', 'movement_type_2', 'Term_CM', 'Currency_2']

# Encode categorical variables using one-hot encoding
encoded_df = pd.get_dummies(df_v2, columns=categorical_cols, drop_first=True)

fcstP_list = ['EOP_CM', 'EOP_LM', 'CIE_VOL', 'VOF_VOL']

for fcst_value in fcstP_list:
    X = encoded_df.drop([fcst_value, 'EOP_CM', 'EOP_LM', 'period', 'TD_type', 'NET', 'CIE_VOL', 'ADB_VOL', 'VOF_VOL', 'NII_VOL'], axis=1)
    y = encoded_df[fcst_value]
    model = xgb.XGBRegressor()
    cv_scores = cross_val_score(model, X, y, cv=15, scoring='r2')
    y_pred = cross_val_predict(model, X, y, cv=15)
    df_v2[f'FCST_{fcst_value}'] = y_pred

df_v2

,period,movement_type,movement_type_2,Term_CM,Currency_2,TD_type,NET,EOP_CM,EOP_LM,CIE_VOL,ADB_VOL,VOF_VOL,NII_VOL,Year,Month,FCST_EOP_CM,FCST_EOP_LM,FCST_CIE_VOL,FCST_VOF_VOL
0,2021-04-01,1UNCHANGED,1UNCHANGED,less than 1M,VND,NOT_ROLL,0.000000,0.470551,0.470551,-110.167907,1.338282e+06,5012.165182,0.0,2021,4,56.526321,50.071621,-196.144882,4640.195801
1,2023-12-01,6ROLL_AUTOROLL,3ROLL,1-3M,VND,AUTOROLL,-340.134811,11656.788337,11996.923148,0.000000,0.000000e+00,0.000000,0.0,2023,12,62.726463,280.529968,-147.134308,110.203842
2,2021-03-01,4PREMATURE,4PREMATURE,less than 1M,VND,NOT_ROLL,-19518.895836,0.000000,19518.895836,-138.625089,1.273054e+06,4929.967179,0.0,2021,3,-10.017500,13426.752930,-282.135986,4755.499023
3,2021-05-01,1UNCHANGED,1UNCHANGED,less than 1M,VND,NOT_ROLL,0.000000,0.558014,0.558014,-116.431460,1.289682e+06,5034.638263,0.0,2021,5,39.018467,50.071621,-73.825012,4649.798340
4,2023-10-01,1NEW_NOTROLL,2NEW,12-18M,VND,NOT_ROLL,167.944246,167.944246,0.000000,-42057.039450,5.545051e+06,49835.134811,0.0,2023,10,180.027786,9.030796,-41738.593750,49494.281250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3585,2019-10-01,6OTHERS,6OTHERS,6-9M,NON-VND,NOT_ROLL,2.592220,195.844490,193.252270,-10.625489,1.772443e+04,44.656294,0.0,2019,10,222.690216,310.746094,-410.435455,53.464680
3586,2019-06-01,6OTHERS,6OTHERS,6-9M,VND,NOT_ROLL,19.475957,59.511124,40.035167,-18051.798267,3.117658e+06,20394.676616,0.0,2019,6,71.878120,52.770870,-18652.320312,21274.310547
3587,2023-03-01,9OTHERS,6OTHERS,12-18M,NON-VND,NOT_ROLL,0.000000,0.000000,0.000000,-1.975563,2.567737e+04,38.249368,0.0,2023,3,153.153336,201.097443,139.448273,-59.006306
3588,2023-03-01,7PREMATURE_AUTOROLL,4PREMATURE,less than 1M,VND,AUTOROLL,-14191.499161,0.000000,14191.499161,-725.396529,1.571298e+06,7719.142902,0.0,2023,3,-4.043190,17321.919922,-1044.313477,9318.589844


In [108]:
# df_grouped.to_csv(r'C:\Users\hiennpd3\OneDrive - VPBank\AA Team\2. TD forecast model\3. Architecture\Package\Model V2\test_data_2.csv')